# Introduction

This notebook shows basic usage of MLFlow run tracking.

In [1]:
import mlflow
import sklearn
import sklearn.datasets
import sklearn.linear_model
import sklearn.metrics
import logging


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("Linear Learner")

# Initialize MLFlow

Make sure to run the following command in a terminal before you proceed.

The following cell checks whether MLFlow is running correctly.

In [2]:
import urllib.request
with urllib.request.urlopen("http://127.0.0.1:5000") as req:
    assert req.status == 200

The next command tells MLFlow where the Tracking Server is.

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Train Model

In [4]:
X, y = sklearn.datasets.load_diabetes(return_X_y=True, as_frame=True)

In [5]:
model = sklearn.linear_model.LinearRegression()
model.fit(X, y)
r2 = model.score(X, y)
logger.info(f"{r2=}")

INFO:Linear Learner:r2=0.5177494254132934


# Store Model in MLFlow

## Option 1: Context manager

In [6]:
MODEL_NAME = "model"
with mlflow.start_run() as run:
    mlflow.sklearn.log_model(model, MODEL_NAME)
    mlflow.log_param("loss", "l2")
    mlflow.log_metric("r2", r2)
    option1_run_id = run.info.run_id
    logger.info(f"{option1_run_id=}")

INFO:Linear Learner:option1_run_id='1cd1ee50e4ef44859e4378fdd7ead694'


## Option 2: Without context manager

In [7]:
run = mlflow.start_run()

In [8]:
MODEL_NAME = "model"
mlflow.sklearn.log_model(model, MODEL_NAME)
mlflow.log_param("loss", "l2")
mlflow.log_metric("r2", r2)
option2_run_id = run.info.run_id
logger.info(f"{option2_run_id=}")

INFO:Linear Learner:option2_run_id='beca1fc597ee4af8b3cf861a01d2b1f3'


In [9]:
mlflow.end_run()

# Reactivate an old run

The following cells add the mean squared error metric to the run in Option 1.

In [10]:
run = mlflow.start_run(run_id=option1_run_id)

In [11]:
logger.info(f"{run.info.run_id=}")

INFO:Linear Learner:run.info.run_id='1cd1ee50e4ef44859e4378fdd7ead694'


In [12]:
mse = sklearn.metrics.mean_squared_error(y, model.predict(X))
logger.info(f"{mse=}")

INFO:Linear Learner:mse=2859.6903987680657


In [13]:
mlflow.log_metric("mse", mse)

In [14]:
mlflow.end_run()